In [4]:
import pandas as pd
import os
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import json

In [5]:
def parse_json_file(json_file):
    """
    Função para converter arquivo JSON de sentimentos em um DataFrame.
    
    Parâmetros:
    json_file -- caminho para o arquivo JSON ou dados JSON em formato de string
    
    Retorno:
    Um DataFrame pandas com duas colunas: sentiment e confidence
    """
    # Lista para armazenar os resultados
    resultados = []
    
    # Verifica se o input é um caminho de arquivo ou uma string JSON
    if isinstance(json_file, str) and (json_file.endswith('.json') or '/' in json_file):
        # Lê o arquivo linha por linha
        with open(json_file, 'r') as file:
            linhas = file.readlines()
    else:
        # Assume que é uma string com linhas JSON
        linhas = json_file.strip().split('\n')
    
    # Processa cada linha
    for linha in linhas:
        try:
            # Carrega o JSON externo
            obj_externo = json.loads(linha)
            
            # Obtém a string de resposta
            resposta_str = obj_externo.get('response', '{}')
            
            # Remove possíveis espaços em branco extras
            resposta_str = resposta_str.strip()
            
            # Carrega o JSON interno
            try:
                # Tenta carregar como objeto JSON
                obj_interno = json.loads(resposta_str)
            except json.JSONDecodeError:
                # Se falhar, pode ser porque o JSON está dentro de uma string
                if resposta_str.startswith('"') and resposta_str.endswith('"'):
                    resposta_str = resposta_str[1:-1]
                obj_interno = json.loads(resposta_str)
            
            # Extrai o sentimento e a confiança
            sentimento = obj_interno.get('sentiment', '')
            confianca = obj_interno.get('confidence', 0.0)
            
            # Normaliza o sentimento para minúsculas para consistência
            sentimento = sentimento.lower()
            
            # Adiciona à lista de resultados
            resultados.append({'sentiment': sentimento, 'confidence': confianca})
            
        except Exception as e:
            print(f"Erro ao processar linha: {linha}")
            print(f"Erro: {e}")
    
    # Cria o DataFrame
    df = pd.DataFrame(resultados)
    
    return df

In [8]:
issues_prs_data = pd.read_csv(r'C:\Users\breno\OneDrive\Documentos\GitHub\stnl-dataminer-api\studycase\data\Issues&PRs.csv')

issues_prs_sentiment = parse_json_file(r'C:\Users\breno\OneDrive\Documentos\GitHub\stnl-dataminer-api\studycase\sentiment\results\Issues&PRs.json')

In [10]:
issues_prs_data.head()

id                     repository   record_id  number  \
0   1  aisepucrio/stnl-dataminer-api  2409854590       4   
1   2  aisepucrio/stnl-dataminer-api  2938799373       5   
2   3  aisepucrio/stnl-dataminer-api  2938763741       3   
3   4  aisepucrio/stnl-dataminer-api  2938748495       2   

                                               title state    creator  \
0  Correção do timestamp do GitHub e melhoria na ...  open  brenonevs   
1  Repositórios privados não podem ser minerados ...  open  brenonevs   
2  Corrigir restrição de domínio na mineração de ...  open  brenonevs   
3         Suporte a autenticação OAuth para o GitHub  open  brenonevs   

  assignees labels  milestone  ...  \
0        []     []        NaN  ...   
1        []     []        NaN  ...   
2        []     []        NaN  ...   
3        []     []        NaN  ...   

                                                body comments timeline_events  \
0  Este PR corrige o formato do timestamp extraíd...       []              []   
1  Atualmente, a API utiliza o PyDriller para ext...       []              []   
2  Atualmente, ao tentar minerar issues do Jira, ...       []              []   
3  Descrição:\n\nAtualmente, a API utiliza tokens...       []              []   

   merged_at commits is_pull_request author_association  \
0        NaN      []            True                NaN   
1        NaN      []           False             MEMBER   
2        NaN      []           False             MEMBER   
3        NaN      []           False             MEMBER   

                                           reactions          tipo  \
0                                                 {}  pull_request   
1  {"+1": 0, "-1": 0, "url": "https://api.github....         issue   
2  {"+1": 0, "-1": 0, "url": "https://api.github....         issue   
3  {"+1": 0, "-1": 0, "url": "https://api.github....         issue   

                      time_mined  
0  2025-03-24 14:00:38.917773+00  
1  2025-03-24 14:04:10.682501+00  
2  2025-03-24 14:04:11.055904+00  
3    2025-03-24 14:04:11.4365+00  

[4 rows x 24 columns]

In [12]:
issues_prs_sentiment.head()

sentiment  confidence
0  positive         0.9
1  negative         0.9
2  negative         0.9
3  positive         0.8

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Dict, Optional
import numpy as np

class IssuesPRsSentimentAnalyzer:
    """Analisador de sentimentos para dados de issues e PRs do GitHub.
    
    Esta classe oferece funcionalidades para analisar e visualizar dados de sentimentos
    associados a issues e pull requests, incluindo distribuições gerais, 
    evolução temporal e análise por tipo de registro.
    
    Attributes:
        COLOR_MAP (Dict[str, str]): Mapeamento de cores para diferentes sentimentos:
            - positive: verde (#2ecc71)
            - neutral: cinza (#95a5a6)
            - negative: vermelho (#e74c3c)
        issues_prs_data (pd.DataFrame): DataFrame contendo os dados de issues e PRs.
        sentiment_df (pd.DataFrame): DataFrame contendo os dados de sentimentos.
        merged_data (pd.DataFrame): DataFrame resultante da mesclagem dos dados e sentimentos.
    """

    COLOR_MAP = {
        'positive': '#2ecc71',
        'neutral': '#95a5a6',
        'negative': '#e74c3c'
    }
    
    def __init__(self, issues_prs_data: pd.DataFrame, issues_prs_sentiment: pd.DataFrame):
        """Inicializa o analisador de sentimentos de issues e PRs.
        
        Args:
            issues_prs_data (pd.DataFrame): DataFrame contendo os dados brutos de issues e PRs.
            issues_prs_sentiment (pd.DataFrame): DataFrame contendo os dados de análise de sentimentos.
        """
        self.issues_prs_data = issues_prs_data
        issues_prs_sentiment['sentiment'] = issues_prs_sentiment['sentiment'].str.lower()
        self.sentiment_df = issues_prs_sentiment
        self.merged_data = self._merge_dataframes()
        
    def _merge_dataframes(self) -> pd.DataFrame:
        """Mescla os DataFrames de issues/PRs e sentimentos.
        
        Combina os dados de issues e PRs com os dados de sentimentos, adicionando colunas de
        sentimento e confiança, além de converter a coluna de data para o formato datetime.
        
        Returns:
            pd.DataFrame: DataFrame mesclado contendo dados de issues/PRs e sentimentos.
        """
        merged = self.issues_prs_data.copy()
        merged['sentiment'] = self.sentiment_df['sentiment']
        merged['confidence'] = self.sentiment_df['confidence']
        merged['time_mined'] = pd.to_datetime(merged['time_mined'])
        return merged

    def plot_overall_sentiment_distribution(self):
        """Gera gráfico de pizza mostrando a distribuição geral de sentimentos.
        
        Cria uma visualização da distribuição geral de sentimentos (positivo, neutro, negativo)
        em todo o conjunto de dados de issues e PRs.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de pizza com a distribuição de sentimentos.
        """
        fig = px.pie(
            self.merged_data,
            names='sentiment',
            title='Distribuição Geral de Sentimentos em Issues e PRs',
            color='sentiment',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            showlegend=True,
            legend_title='Sentimentos'
        )
        return fig

    def plot_sentiment_by_record_type(self):
        """Gera gráfico de barras agrupadas mostrando sentimentos por tipo de registro (Issue/PR).
        
        Cria uma visualização da distribuição de sentimentos segmentada por tipo de registro
        (issue ou pull request).
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com distribuição de sentimentos por tipo.
        """
        sentiment_by_type = pd.crosstab(
            self.merged_data['tipo'],
            self.merged_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_type,
            title='Distribuição de Sentimentos por Tipo de Registro',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Tipo',
            yaxis_title='Quantidade',
            legend_title='Sentimento'
        )
        return fig

    def plot_temporal_sentiment_evolution(self):
        """Gera gráfico de linha mostrando a evolução dos sentimentos ao longo do tempo.
        
        Cria uma visualização da evolução temporal dos diferentes sentimentos (positivo, 
        neutro, negativo) ao longo dos meses.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução temporal dos sentimentos.
        """
        temporal_analysis = (
            self.merged_data
            .groupby([pd.Grouper(key='time_mined', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP.get(sentiment, '#7f7f7f'), width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title='Evolução Temporal dos Sentimentos',
            xaxis_title='Data',
            yaxis_title='Quantidade de Registros',
            hovermode='x unified'
        )
        return fig

    def plot_creator_sentiment_evolution(self, creator: str):
        """Gera gráfico de linha mostrando a evolução dos sentimentos para um criador específico.
        
        Args:
            creator (str): Nome do criador para análise.
            
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com evolução dos sentimentos do criador.
        """
        creator_data = self.merged_data[self.merged_data['creator'] == creator]
        
        temporal_analysis = (
            creator_data
            .groupby([pd.Grouper(key='time_mined', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        fig = go.Figure()
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP.get(sentiment, '#7f7f7f'), width=2),
                    marker=dict(size=6)
                )
            )
        
        fig.update_layout(
            title=f'Evolução de Sentimentos para {creator}',
            xaxis_title='Data',
            yaxis_title='Quantidade de Registros',
            hovermode='x unified'
        )
        return fig

    def plot_sentiment_by_state(self):
        """Gera gráfico de barras mostrando a distribuição de sentimentos por estado (open/closed).
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com sentimentos por estado.
        """
        sentiment_by_state = pd.crosstab(
            self.merged_data['state'],
            self.merged_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_state,
            title='Distribuição de Sentimentos por Estado',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Estado',
            yaxis_title='Quantidade',
            legend_title='Sentimento'
        )
        return fig

    def plot_sentiment_by_repository(self):
        """Gera gráfico de barras mostrando sentimentos por repositório.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com sentimentos por repositório.
        """
        sentiment_by_repo = pd.crosstab(
            self.merged_data['repository'],
            self.merged_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_repo,
            title='Distribuição de Sentimentos por Repositório',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Repositório',
            yaxis_title='Quantidade',
            legend_title='Sentimento',
            xaxis_tickangle=45
        )
        return fig

    def plot_confidence_distribution(self):
        """Gera histograma da distribuição de confiança nas análises de sentimentos.
        
        Returns:
            plotly.graph_objs._figure.Figure: Histograma de confiança.
        """
        fig = px.histogram(
            self.merged_data, 
            x='confidence',
            color='sentiment',
            title='Distribuição de Confiança por Sentimento',
            color_discrete_map=self.COLOR_MAP,
            nbins=10
        )
        fig.update_layout(
            xaxis_title='Confiança',
            yaxis_title='Contagem',
            bargap=0.1
        )
        return fig

    def plot_sentiment_count_by_creator(self, top_n: int = 10):
        """Gera gráfico de barras dos sentimentos por criador para os top_n criadores mais ativos.
        
        Args:
            top_n (int): Número de criadores mais ativos a serem mostrados.
            
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras com sentimentos por criador.
        """
        # Contar o número total de registros por criador
        creator_counts = self.merged_data['creator'].value_counts().reset_index()
        creator_counts.columns = ['creator', 'count']
        
        # Selecionar os top_n criadores mais ativos
        top_creators = creator_counts.head(top_n)['creator'].tolist()
        
        # Filtrar dados apenas para esses criadores
        filtered_data = self.merged_data[self.merged_data['creator'].isin(top_creators)]
        
        # Criar contagem cruzada
        sentiment_by_creator = pd.crosstab(
            filtered_data['creator'],
            filtered_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_creator,
            title=f'Distribuição de Sentimentos por Criador (Top {top_n})',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Criador',
            yaxis_title='Quantidade',
            legend_title='Sentimento',
            xaxis_tickangle=45
        )
        return fig

    def plot_timeline_activity_with_sentiment(self):
        """Gera gráfico mostrando atividade ao longo do tempo com cores de sentimento.
        
        Creates a visualization that shows the timeline of issues and PRs activity with 
        sentiment indicated by colors.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de linha com atividade e sentimentos.
        """
        # Agregação temporal dos dados com contagem de sentimentos
        temporal_data = (
            self.merged_data
            .groupby([pd.Grouper(key='time_mined', freq='M'), 'tipo'])
            .agg({
                'id': 'count',
                'sentiment': lambda x: (x == 'positive').mean()  # Proporção de sentimentos positivos
            })
            .reset_index()
        )
        
        fig = px.scatter(
            temporal_data,
            x='time_mined',
            y='id',
            size='id',
            color='sentiment',
            facet_row='tipo',
            title='Atividade Temporal com Indicação de Sentimento',
            color_continuous_scale='RdYlGn',
            labels={
                'time_mined': 'Data',
                'id': 'Quantidade',
                'sentiment': 'Sentimento Positivo (%)'
            }
        )
        
        fig.update_layout(
            height=600,
            hovermode='closest'
        )
        
        fig.update_traces(
            hovertemplate='<b>%{x}</b><br>' +
                         'Quantidade: %{y}<br>' +
                         'Sentimento Positivo: %{marker.color:.1%}'
        )
        
        return fig

    def plot_sentiment_by_pull_request_status(self):
        """Gera gráfico comparando sentimentos entre PRs mesclados e não mesclados.
        
        Filtra apenas PRs e compara a distribuição de sentimentos entre aqueles que foram
        mesclados e os que não foram.
        
        Returns:
            plotly.graph_objs._figure.Figure: Gráfico de barras comparando sentimentos por status de PR.
        """
        # Filtrar apenas PRs
        prs_data = self.merged_data[self.merged_data['is_pull_request'] == True].copy()
        
        # Criar coluna de status do PR (mesclado ou não)
        prs_data['merged_status'] = prs_data['merged_at'].apply(
            lambda x: 'Mesclado' if pd.notna(x) else 'Não Mesclado'
        )
        
        # Cálculo das contagens
        sentiment_by_status = pd.crosstab(
            prs_data['merged_status'],
            prs_data['sentiment']
        )
        
        fig = px.bar(
            sentiment_by_status,
            title='Distribuição de Sentimentos por Status de PR',
            barmode='group',
            color_discrete_map=self.COLOR_MAP
        )
        fig.update_layout(
            xaxis_title='Status do PR',
            yaxis_title='Quantidade',
            legend_title='Sentimento'
        )
        return fig

    def plot_dashboard(self):
        """Gera um dashboard com múltiplos gráficos para análise de sentimentos.
        
        Returns:
            plotly.graph_objs._figure.Figure: Dashboard com múltiplos gráficos.
        """
        fig = make_subplots(
            rows=2, 
            cols=2,
            subplot_titles=(
                'Distribuição Geral de Sentimentos',
                'Sentimentos por Tipo',
                'Evolução Temporal de Sentimentos',
                'Sentimentos por Estado'
            ),
            specs=[
                [{"type": "pie"}, {"type": "bar"}],
                [{"type": "scatter"}, {"type": "bar"}]
            ]
        )
        
        # 1. Gráfico de pizza com distribuição geral
        sentiment_counts = self.merged_data['sentiment'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=sentiment_counts.index,
                values=sentiment_counts.values,
                marker=dict(colors=[self.COLOR_MAP.get(s, '#7f7f7f') for s in sentiment_counts.index])
            ),
            row=1, col=1
        )
        
        # 2. Gráfico de barras por tipo
        sentiment_by_type = pd.crosstab(
            self.merged_data['tipo'],
            self.merged_data['sentiment']
        )
        
        for sentiment in sentiment_by_type.columns:
            fig.add_trace(
                go.Bar(
                    x=sentiment_by_type.index,
                    y=sentiment_by_type[sentiment],
                    name=sentiment,
                    marker_color=self.COLOR_MAP.get(sentiment, '#7f7f7f')
                ),
                row=1, col=2
            )
        
        # 3. Evolução temporal
        temporal_analysis = (
            self.merged_data
            .groupby([pd.Grouper(key='time_mined', freq='M'), 'sentiment'])
            .size()
            .unstack(fill_value=0)
        )
        
        for sentiment in temporal_analysis.columns:
            fig.add_trace(
                go.Scatter(
                    x=temporal_analysis.index,
                    y=temporal_analysis[sentiment],
                    name=sentiment,
                    mode='lines+markers',
                    line=dict(color=self.COLOR_MAP.get(sentiment, '#7f7f7f'))
                ),
                row=2, col=1
            )
        
        # 4. Sentimentos por estado
        sentiment_by_state = pd.crosstab(
            self.merged_data['state'],
            self.merged_data['sentiment']
        )
        
        for sentiment in sentiment_by_state.columns:
            fig.add_trace(
                go.Bar(
                    x=sentiment_by_state.index,
                    y=sentiment_by_state[sentiment],
                    name=sentiment,
                    marker_color=self.COLOR_MAP.get(sentiment, '#7f7f7f')
                ),
                row=2, col=2
            )
        
        fig.update_layout(
            height=800,
            title_text='Dashboard de Análise de Sentimentos em Issues e PRs',
            showlegend=True
        )
        
        return fig

In [14]:
issues_prs_analysis = IssuesPRsSentimentAnalyzer(issues_prs_data, issues_prs_sentiment)

In [20]:
fig1 = issues_prs_analysis.plot_overall_sentiment_distribution()
fig1.show()

fig2 = issues_prs_analysis.plot_sentiment_by_record_type()
fig2.show()

fig3 = issues_prs_analysis.plot_temporal_sentiment_evolution()
fig3.show()

fig4 = issues_prs_analysis.plot_sentiment_by_state()
fig4.show()

fig5 = issues_prs_analysis.plot_sentiment_by_repository()
fig5.show()

fig6 = issues_prs_analysis.plot_confidence_distribution()
fig6.show()

fig7 = issues_prs_analysis.plot_sentiment_count_by_creator()
fig7.show()

fig8 = issues_prs_analysis.plot_timeline_activity_with_sentiment()
fig8.show()

fig9 = issues_prs_analysis.plot_sentiment_by_pull_request_status()
fig9.show()

fig10 = issues_prs_analysis.plot_dashboard()
fig10.show()













C:\Users\breno\AppData\Local\Temp\ipykernel_13764\3104835849.py:118: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\breno\AppData\Local\Temp\ipykernel_13764\3104835849.py:306: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



C:\Users\breno\AppData\Local\Temp\ipykernel_13764\3104835849.py:431: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

